In [1]:
import os
os.getcwd()

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scanpy as sc
import anndata as ad
sc.settings.verbosity = 0

In [2]:
import GenKI as gk
from GenKI.preprocesing import build_adata
from GenKI.dataLoader import DataLoader
from GenKI.train import VGAE_trainer
from GenKI import utils

%load_ext autoreload
%autoreload 2

/home/g0bi/miniforge3/envs/ogenki/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/home/g0bi/miniforge3/envs/ogenki/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/home/g0bi/miniforge3/envs/ogenki/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_hdf from `anndata` is deprecated. Import anndata.io.read_hdf instead.
  warnings.warn(msg, FutureWarning)
/home/g0bi/miniforge3/envs/ogenki/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_loom from `anndata` is deprecated. Import anndata.io.read_loom instead.
  warnings.warn(msg, FutureWarning)
/home/g0bi/miniforge3/envs/ogenki/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing 

In [3]:
# subset data
adata = build_adata("../data/adata.h5ad")
#adata = adata[:120, :300].copy()
adata = adata[:1200, :3000].copy()
adata

load counts from ../data/adata.h5ad


AnnData object with n_obs × n_vars = 2313 × 5499
    obs: 'VSN_cell_type', 'VSN_leiden_res0.3', 'VSN_leiden_res0.6', 'VSN_leiden_res0.9', 'VSN_leiden_res1.2', 'VSN_sample_id', 'Seurat_leiden_res0.6', 'Seurat_leiden_res1.2', 'Seurat_cell_type', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt'
    var: 'gene_ids', 'feature_types', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'Seurat_cell_type_colors', 'hvg', 'log1p', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'norm'
    obsp: 'connectivities', 'distances'

In [4]:
adata.obs

,VSN_cell_type,VSN_leiden_res0.3,VSN_leiden_res0.6,VSN_leiden_res0.9,VSN_leiden_res1.2,VSN_sample_id,Seurat_leiden_res0.6,Seurat_leiden_res1.2,Seurat_cell_type,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt
CTTTGGGAGTCAATTG-1,MOL_A,MOL_A (1),MOL_A_2 (1),MOL_A_1 (0),MOL_A_2 (0),10x_multiome_brain,MOL (0),MOL (0),MOL,1926,4351.0,5.0,0.114916
TCAGGAGCATAGACCC-1,MOL_A,MOL_A (1),MOL_A_2 (1),MOL_A_1 (0),MOL_A_2 (0),10x_multiome_brain,NFOL (1),MOL (1),MOL,2607,6673.0,8.0,0.119886
TATAGGTGTTTGGCGG-1,MOL_A,MOL_A (1),MOL_A_2 (1),MOL_A_1 (0),MOL_A_2 (0),10x_multiome_brain,MOL (0),MOL (1),MOL,2095,4979.0,2.0,0.040169
TAGTCAATCGTACCGA-1,MOL_B,MOL_B (0),MOL_B_1 (0),MOL_B_1 (1),MOL_B_4 (4),10x_multiome_brain,NFOL (1),NFOL (3),NFOL,2829,10526.0,2.0,0.019001
AGCTACTAGGAGGTTA-1,GC,GC (4),GC (5),GC (6),GC (8),10x_multiome_brain,GC (4),GC (5),GC,4132,10621.0,11.0,0.103568
...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTCAAAGGGACGCA-1,AST_CER,AST_CER (2),AST_CER (2),AST_CER (2),AST_CER_2 (5),10x_multiome_brain,BG (2),BG (2),BG,2457,4817.0,1.0,0.020760
TTTGGCTGTTACAACG-1,MOL_A,MOL_A (1),MOL_A_2 (1),MOL_A_1 (0),MOL_A_2 (0),10x_multiome_brain,MOL (0),MOL (0),MOL,2072,5152.0,4.0,0.077640
TTTAGCAAGGTTAGAG-1,AST_CER,AST_CER (2),AST_CER (2),AST_CER (2),AST_CER_1 (7),10x_multiome_brain,BG (2),BG (2),BG,2116,3767.0,7.0,0.185824
TCATGTTTCACAGCGC-1,MOL_B,MOL_B (0),MOL_B_1 (0),MOL_B_1 (1),MOL_B_1 (1),10x_multiome_brain,MOL (0),MOL (1),MOL,2953,12020.0,7.0,0.058236


In [5]:
adata.var
adata.var_names

Index(['LINC01409', 'HES4', 'ISG15', 'C1ORF159', 'AL645728.1', 'CDK11B', 'SKI',
       'PLCH2', 'HES5', 'AL139246.5',
       ...
       'MT-CO1', 'MT-CO2', 'MT-ATP8', 'MT-ATP6', 'MT-CO3', 'MT-ND3', 'MT-ND4L',
       'MT-ND6', 'AC007325.4', 'AC007325.2'],
      dtype='object', length=5499)

In [6]:
# marker genes
marker_genes =  ["SIAH1", "PRIM2", "DYNC1H1", "SIRT1", "PPP1CA", "PDE6D", "YWHAZ", "FBXO42", "CHMP5", 
    "SNRPB", "CRK", "TREH",  "UQCRC2", "PUM2", "SNRPD2", "CCT5", "KAT5", "LYN", 
    "PRKCE", "HAGH", "DNAJA1", "DNAJA2", "SNCA", "TH", "KIT", "KITLG", "NR4A2"]

marker_genes = set(marker_genes)
marker_genes_found=[]

for i in marker_genes:
    if i in adata.var_names:
        marker_genes_found.append(i)
        print(i+" found in the dataset")
    else:
        print(i+" not found in the dataset")
marker_genes_found

SIRT1 not found in the dataset
UQCRC2 not found in the dataset
FBXO42 not found in the dataset
SIAH1 not found in the dataset
DNAJA2 not found in the dataset
PRIM2 not found in the dataset
CHMP5 not found in the dataset
SNRPD2 not found in the dataset
PRKCE not found in the dataset
KITLG found in the dataset
PDE6D not found in the dataset
SNRPB not found in the dataset
HAGH not found in the dataset
YWHAZ not found in the dataset
SNCA found in the dataset
TH not found in the dataset
NR4A2 found in the dataset
PUM2 not found in the dataset
KIT found in the dataset
PPP1CA not found in the dataset
CCT5 found in the dataset
KAT5 not found in the dataset
CRK not found in the dataset
LYN found in the dataset
DNAJA1 found in the dataset
TREH not found in the dataset
DYNC1H1 not found in the dataset


['KITLG', 'SNCA', 'NR4A2', 'KIT', 'CCT5', 'LYN', 'DNAJA1']

In [ ]:
# load data
data_wrapper =  DataLoader(
                adata, # adata object
                target_gene = ["HES4"], # KO gene name
                target_cell = None, # obsname for cell type, if none use all
                obs_label = "gene_ids", # colname for genes
                GRN_file_dir = "GRNs", # folder name for GRNs
                rebuild_GRN = True, # whether build GRN by pcNet
                pcNet_name = "pcNet_scenic", # GRN file name
                verbose = True, # whether verbose
                n_cpus = 8, # multiprocessing
                )

data_wt = data_wrapper.load_data()
data_ko = data_wrapper.load_kodata()

use all the cells (2313) in adata
build GRN


2025-04-21 18:51:37,281	INFO worker.py:1841 -- Started a local Ray instance.


ray init, using 8 CPUs


In [ ]:
# init trainer
hyperparams = {"epochs": 100, 
               "lr": 7e-4, 
               "beta": 1e-4, 
               "seed": 8096}
log_dir=None 

sensei = VGAE_trainer(data_wt, 
                     epochs=hyperparams["epochs"], 
                     lr=hyperparams["lr"], 
                     log_dir=log_dir, 
                     beta=hyperparams["beta"],
                     seed=hyperparams["seed"],
                     verbose=False,
                     )

In [ ]:
# %%timeit
sensei.train()

In [ ]:
sensei.save_model('adata-save')

In [ ]:
# get distance between wt and ko
z_mu_wt, z_std_wt = sensei.get_latent_vars(data_wt)
z_mu_ko, z_std_ko = sensei.get_latent_vars(data_ko)
dis = gk.utils.get_distance(z_mu_ko, z_std_ko, z_mu_wt, z_std_wt, by="KL")
print(dis.shape)

In [ ]:
# raw ranked gene list
res_raw = utils.get_generank(data_wt, dis, rank=True)
res_raw.head()

In [ ]:
# if permutation test
null = sensei.pmt(data_ko, n=100, by="KL")
res = utils.get_generank(data_wt, dis, null,)

# save_significant_as = 'gene_list_example')
res